In [1]:
import os
import numpy as np
import cv2
import glob
import string
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Reshape, Input, Dropout,
    Bidirectional, LSTM, BatchNormalization, TimeDistributed, Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# ✅ SET PATHS
download_folder = r"C:\Users\nepal_refipgc\Downloads\Advance Project"
dataset_path = os.path.join(download_folder, "dataset_Ravin")
label_file = os.path.join(dataset_path, "IAM", "gt_test.txt")
image_folder = os.path.join(dataset_path, "IAM", "image")  # Ensure images are here

# ✅ LOAD LABEL FILE
labels_dict = {}
with open(label_file, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            filename, text = parts
            labels_dict[filename.split('.')[0]] = text  # Remove .png/.jpg for matching

print(f"✅ Loaded {len(labels_dict)} labels from gt_test.txt")

# ✅ LOAD IMAGE FILES
image_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(image_folder)
    for file in files
    if file.lower().endswith(('.png', '.jpg', '.jpeg'))
]

# ✅ MATCH IMAGE FILES WITH LABELS
filtered_images, filtered_labels = [], []
for img_path in image_files:
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    if img_name in labels_dict:
        filtered_images.append(img_path)
        filtered_labels.append(labels_dict[img_name])

print(f"✅ Matched {len(filtered_images)} images with labels.")
print(f"❌ Unmatched images: {len(image_files) - len(filtered_images)}")
print(f"📊 Ready dataset size: {len(filtered_images)} image-label pairs.")

# ✅ IMAGE PREPROCESSING FUNCTION
def preprocess_image(image_path, img_size=(128, 32)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_size[1], img_size[0]))  # Ensure correct (Width, Height)
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=-1)  # Add channel dimension for CNN input
    return img

# ✅ PREPROCESS IMAGES
X_data = np.array([preprocess_image(img) for img in filtered_images])
print(f"✅ Image shape after preprocessing: {X_data.shape}")  # Expected: (num_samples, 128, 32, 1)

# ✅ CHARACTER MAPPING FOR TEXT LABELS
all_characters = string.ascii_letters + string.digits + string.punctuation + " "
num_classes = len(all_characters) + 1  # +1 for CTC blank token

char_to_int = {ch: i for i, ch in enumerate(all_characters)}
char_to_int["<BLANK>"] = num_classes - 1  # CTC Blank at the end
int_to_char = {i: ch for ch, i in char_to_int.items()}

# ✅ CONVERT TEXT LABELS INTO INTEGER SEQUENCES
label_sequences = [
    [char_to_int.get(char, char_to_int[" "]) for char in label]  # Map unknown chars to space
    for label in filtered_labels
]

# ✅ PAD LABEL SEQUENCES
max_label_length = max(len(seq) for seq in label_sequences)
y_data = pad_sequences(label_sequences, maxlen=max_label_length, padding="post", value=char_to_int["<BLANK>"])

# ✅ TRAIN-TEST SPLIT
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print(f"✅ Training Data: {X_train.shape}, Labels: {y_train.shape}")
print(f"✅ Validation Data: {X_val.shape}, Labels: {y_val.shape}")

# ✅ COMPUTE INPUT LENGTHS FOR CTC LOSS
train_input_length = np.full((len(X_train), 1), X_train.shape[2] // 4)
val_input_length = np.full((len(X_val), 1), X_val.shape[2] // 4)

train_label_length = np.array([[len(label[label != char_to_int["<BLANK>"]])] for label in y_train])
val_label_length = np.array([[len(label[label != char_to_int["<BLANK>"]])] for label in y_val])

# ✅ DEFINE CTC LOSS FUNCTION
def ctc_loss_lambda(args):
    y_true, y_pred, input_length, label_length = args
    return K.ctc_batch_cost(y_true, y_pred, input_length, label_length)

# ✅ BUILD MODEL
def build_model(input_shape=(128, 32, 1), output_units=num_classes):
    inputs = Input(shape=input_shape, name="image_input")

    # CNN Feature Extraction
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Reshape for LSTM
    x = Reshape(target_shape=(X_train.shape[2] // 4, -1))(x)  # ✅ FIXED OUTPUT SHAPE
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3))(x)
    x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.3))(x)

    # Fully Connected Output
    outputs = Dense(output_units, activation="softmax", name="predictions")(x)

    # Define additional inputs
    labels = Input(name="labels", shape=(None,), dtype="int32")
    input_length = Input(name="input_length", shape=(1,), dtype="int32")
    label_length = Input(name="label_length", shape=(1,), dtype="int32")

    # CTC loss layer
    loss_out = Lambda(ctc_loss_lambda, output_shape=(1,), name="ctc_loss")(
        [labels, outputs, input_length, label_length]
    )

    model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)
    return model

# ✅ BUILD & COMPILE MODEL
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss={"ctc_loss": lambda y_true, y_pred: y_pred})  # 👈 FIXED

print("✅ Model compiled successfully!")

# ✅ TRAIN MODEL WITH EARLY STOPPING
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(
    {"image_input": X_train, "labels": y_train, "input_length": train_input_length, "label_length": train_label_length},
    np.zeros((len(X_train),)),  # Dummy target for CTC loss
    batch_size=32,
    epochs=100,
    validation_data=(
        {"image_input": X_val, "labels": y_val, "input_length": val_input_length, "label_length": val_label_length},
        np.zeros((len(X_val),))
    ),
    callbacks=[early_stopping],
    verbose=1
)

print("✅ Training complete! 🚀")


✅ Loaded 2915 labels from gt_test.txt
✅ Matched 2915 images with labels.
❌ Unmatched images: 0
📊 Ready dataset size: 2915 image-label pairs.
✅ Image shape after preprocessing: (2915, 128, 32, 1)
✅ Training Data: (2332, 128, 32, 1), Labels: (2332, 93)
✅ Validation Data: (583, 128, 32, 1), Labels: (583, 93)
✅ Model compiled successfully!
Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'model/ctc_loss/CTCLoss' defined at (most recent call last):
    File "C:\Users\nepal_refipgc\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\nepal_refipgc\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Protfolio\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Protfolio\.venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Protfolio\.venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Protfolio\.venv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\nepal_refipgc\AppData\Local\Programs\Python\